In [2]:
import os
import numpy as np
import xarray as xr
import pandas as pd
import netCDF4 as nc
import geopandas as gpd
import rioxarray as rxr
from myfunc import timer
from myfunc import DirMan
from myfunc import load_and_flatten_data
import config

resolution     = config.resolution
region         = config.region
data_path      = config.data_path
post_data_path = config.post_data_path
shp_path       = config.shp_path
fig_path       = config.fig_path

dir_man = DirMan(data_path)
dir_man.enter()

os.makedirs(f'{data_path}/csv', exist_ok=True)

area_sum = 19.925158940748766

Entered /tera11/zhwei/students/Xionghui/data/run/500/


In [3]:
df = pd.DataFrame()

with nc.Dataset('mask123.nc4') as dataset:
    lat = dataset['lat'][:].flatten()
    lon = dataset['lon'][:].flatten()
    df['lat'] = np.repeat(lat, len(lon))
    df['lon'] = np.tile(lon, len(lat))

file_variable_list = [
    ('mask123', 'Band1'),
    ('Sbedrock', 'Sbedrock'),
    ('Area', 'area'),

    ('IGBP', 'LC', 0),
]

for entry in file_variable_list:
    file = entry[0]
    variable_name = entry[1]  
    index = entry[2:] if len(entry) > 2 else None  
    if index:
        df[file] = load_and_flatten_data(file, variable_name, index[0])
    else:
        df[file] = load_and_flatten_data(file, variable_name)

print(df[df['IGBP']<10]['Area'].sum())

df = df.dropna()
df = df[df['Sbedrock'] > 0]
df = df[df['mask123'] == 1]


print(df['Area'].sum())

# df.drop(labels='Area',axis=1,inplace=True)
df.drop(labels='mask123',axis=1,inplace=True)
df.drop(labels='Sbedrock',axis=1,inplace=True)
# with open('csv/Global_statistics.csv','w') as f:
#     df.to_csv(f)
df


63894436398799.51
19925158940748.766


,lat,lon,Area,IGBP
709889414,-55.764582,-67.439577,120773.879698,9
709889415,-55.764582,-67.435411,120773.879698,9
709889416,-55.764582,-67.431244,120773.879698,9
709889417,-55.764582,-67.427077,120773.879698,9
709975814,-55.760415,-67.439577,120786.785891,8
...,...,...,...,...
3586500782,82.960426,-26.739575,26309.225697,1
3586500783,82.960426,-26.735409,26309.225697,1
3586500784,82.960426,-26.731242,26309.225697,1
3586500785,82.960426,-26.727075,26309.225697,1


In [ ]:
df = pd.DataFrame()

with nc.Dataset('Sbedrock.nc4') as dataset:
    lat = dataset['lat'][:].flatten()
    lon = dataset['lon'][:].flatten()
    df['lat'] = np.repeat(lat, len(lon))
    df['lon'] = np.tile(lon, len(lat))

file_variable_list = [
    ('Sbedrock', 'Sbedrock'),
    ('mask123', 'Band1'),
    ('Area', 'area'),
]

for entry in file_variable_list:
    file = entry[0]
    variable_name = entry[1]  
    index = entry[2:] if len(entry) > 2 else None  
    if index:
        df[file] = load_and_flatten_data(file, variable_name, index[0])
    else:
        df[file] = load_and_flatten_data(file, variable_name)

shp2 = gpd.read_file(shp_path+'World_CN/ne_10m_admin_0_countries_chn.shp')
df = df.reset_index(drop=True)
gdf_points = gpd.GeoDataFrame(geometry=gpd.points_from_xy(df['lon'], df['lat']), crs='EPSG:4326')
result2 = gpd.sjoin(gdf_points, shp2, how='left', predicate='within')

df['Sovereignt_short'] = result2['ISO_A3']
df = df[df['Sovereignt_short']=='Brazil']

print(df['Area'].sum()/(1e12))

df = df.dropna()
df = df[df['Sbedrock'] > 0]
df = df[df['mask123'] == 1]
df.drop(labels='mask123',axis=1,inplace=True)

print(df['Area'].sum()/(1e12))

# with open('csv/Global.csv','w') as f:
#     df.to_csv(f)

# df1 = df[df['Sovereignt_short'] == 'USA']
# # print(df1)

# with open('csv/US.csv','w') as f:
#     df1.to_csv(f)

# df1
df